In [1]:
import cv2
import os
import numpy as np
import pandas as pd
from datetime import datetime
import tkinter as tk
from tkinter import Tk, Label, Entry, Button, filedialog, messagebox, Frame, ttk
import shutil
import threading
import queue
import time
import csv
from ultralytics import YOLO
from picamera import PiCamera
from picamera.array import PiRGBArray

ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# Constants
DATASET_PATH = 'dataset'
TRAINER_PATH = 'trainer'
MODEL_FILE = os.path.join(TRAINER_PATH, 'trainer.yml')
ATTENDANCE_FILE = 'attendance.csv'
LOG_FILE = 'system_log.txt'
FRAME_QUEUE_SIZE = 5
DEFAULT_CONFIDENCE_THRESHOLD = 85
DEFAULT_NUM_SAMPLES = 10

# Ensure directories exist
os.makedirs(DATASET_PATH, exist_ok=True)
os.makedirs(TRAINER_PATH, exist_ok=True)

# Initialize YOLO face detection
yolo_model = YOLO('face_yolov8n.pt')

In [ ]:
def log_activity(message):
    """Log activity with timestamp"""
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_message = f"[{timestamp}] {message}\n"
    try:
        with open(LOG_FILE, 'a') as f:
            f.write(log_message)
    except Exception as e:
        print(f"Log write error: {e}")

In [ ]:
class CSVAttendanceSystem:
    def __init__(self, dataset_path=DATASET_PATH, model_file=MODEL_FILE, attendance_file=ATTENDANCE_FILE):
        self.DATASET_PATH = dataset_path
        self.MODEL_FILE = model_file
        self.ATTENDANCE_FILE = attendance_file
        self.LOG_FILE = LOG_FILE
        self.DEFAULT_NUM_SAMPLES = DEFAULT_NUM_SAMPLES
        self.DEFAULT_CONFIDENCE_THRESHOLD = DEFAULT_CONFIDENCE_THRESHOLD
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()
        self.yolo_model = yolo_model
        self.labels = {}
        self.confidence_threshold = DEFAULT_CONFIDENCE_THRESHOLD
        self.status_callback = None
        self.frame_queue = queue.Queue(maxsize=FRAME_QUEUE_SIZE)
        self.attendance_thread = None
        self.stop_flag = False

    def set_status_callback(self, callback):
        self.status_callback = callback

    def update_status(self, message):
        if self.status_callback:
            self.status_callback(message)
        log_activity(message)

    def collect_training_data(self):
        faces, ids = [], []
        label_id = 0
        self.labels = {}

        if not os.path.exists(self.DATASET_PATH):
            self.update_status("[ERROR] Dataset path does not exist")
            return faces, ids

        for person_name in os.listdir(self.DATASET_PATH):
            person_dir = os.path.join(self.DATASET_PATH, person_name)
            if not os.path.isdir(person_dir):
                continue

            self.labels[label_id] = person_name
            image_count = 0

            for img_file in os.listdir(person_dir):
                if not img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    continue

                img_path = os.path.join(person_dir, img_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    self.update_status(f"[WARNING] Could not read image: {img_path}")
                    continue

                img = cv2.resize(img, (100, 100))
                faces.append(img)
                ids.append(label_id)
                image_count += 1

            self.update_status(f"[INFO] Collected {image_count} images for {person_name}")
            label_id += 1

        try:
            with open(os.path.join(TRAINER_PATH, 'labels.csv'), 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['ID', 'Name'])
                for lid, name in self.labels.items():
                    writer.writerow([lid, name])
        except Exception as e:
            self.update_status(f"[ERROR] Failed to save labels: {e}")

        return faces, ids

    def train_model(self):
        self.update_status("[INFO] Training model...")
        faces, ids = self.collect_training_data()

        if not faces:
            self.update_status("[ERROR] No training data found")
            return False

        try:
            self.recognizer.train(faces, np.array(ids))
            self.recognizer.save(self.MODEL_FILE)
            self.update_status(f"[INFO] Model trained with {len(faces)} images")
            return True
        except Exception as e:
            self.update_status(f"[ERROR] Training failed: {e}")
            return False

    def load_model(self):
        try:
            if not os.path.exists(self.MODEL_FILE):
                self.update_status("[ERROR] Model file not found")
                return False

            self.recognizer.read(self.MODEL_FILE)
            self.labels = {}
            labels_file = os.path.join(TRAINER_PATH, 'labels.csv')

            if os.path.exists(labels_file):
                df = pd.read_csv(labels_file)
                self.labels = dict(zip(df['ID'], df['Name']))
                self.update_status("[INFO] Model and labels loaded")
                return True
            else:
                self.update_status("[ERROR] Labels file not found")
                return False
        except Exception as e:
            self.update_status(f"[ERROR] Failed to load model: {e}")
            return False

    def mark_attendance(self, name):
        now = datetime.now()
        date, time_str = now.strftime('%Y-%m-%d'), now.strftime('%H:%M:%S')
        try:
            df = pd.DataFrame([[name, date, time_str]], columns=['Name', 'Date', 'Time'])
            if os.path.exists(self.ATTENDANCE_FILE):
                existing_df = pd.read_csv(self.ATTENDANCE_FILE)
                if not existing_df[(existing_df['Name'] == name) & (existing_df['Date'] == date)].empty:
                    self.update_status(f"[INFO] {name} already marked today")
                    return False
                df.to_csv(self.ATTENDANCE_FILE, mode='a', header=False, index=False)
            else:
                df.to_csv(self.ATTENDANCE_FILE, index=False)
            self.update_status(f"[INFO] Marked attendance for {name}")
            return True
        except Exception as e:
            self.update_status(f"[ERROR] Attendance marking failed: {e}")
            return False

    def set_confidence_threshold(self, threshold):
        try:
            threshold = float(threshold)
            if 0 <= threshold <= 100:
                self.confidence_threshold = threshold
                self.update_status(f"[INFO] Threshold set to {threshold}")
                return True
            self.update_status("[ERROR] Threshold must be 0-100")
            return False
        except ValueError:
            self.update_status("[ERROR] Invalid threshold value")
            return False

    def test_recognition(self, image_path):
        self.update_status("[INFO] Testing recognition...")
        if not self.load_model():
            self.update_status("[ERROR] Model not loaded")
            return

        img = cv2.imread(image_path)
        if img is None:
            self.update_status(f"[ERROR] Could not read image: {image_path}")
            return

        if img.shape[0] > 480 or img.shape[1] > 640:
            img = cv2.resize(img, (320, 240))

        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        gray = clahe.apply(gray)

        results = self.yolo_model(rgb_img, conf=0.3, iou=0.7)
        if not results or not results[0].boxes:
            self.update_status("[WARNING] No face detected")
            return

        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            face_gray = gray[y1:y2, x1:x2]
            if face_gray.size == 0 or face_gray.shape[0] < 10 or face_gray.shape[1] < 10:
                self.update_status("[WARNING] Invalid face region")
                continue
            face_gray = cv2.resize(face_gray, (100, 100))
            try:
                label_id, confidence = self.recognizer.predict(face_gray)
                name = self.labels.get(label_id, "Unknown")
                self.update_status(f"[INFO] Recognized: {name}, Confidence: {confidence}")
            except Exception as e:
                self.update_status(f"[ERROR] Recognition error: {e}")

    def run_real_time_attendance(self):
        if self.attendance_thread and self.attendance_thread.is_alive():
            self.update_status("[INFO] Attendance monitoring already running")
            return

        self.stop_flag = False
        self.attendance_thread = threading.Thread(target=self._attendance_thread)
        self.attendance_thread.daemon = True
        self.attendance_thread.start()
        self.update_status("[INFO] Started real-time attendance")

    def stop_attendance(self):
        if self.attendance_thread and self.attendance_thread.is_alive():
            self.stop_flag = True
            self.update_status("[INFO] Stopping attendance...")
        else:
            self.update_status("[INFO] Attendance not running")

    def _attendance_thread(self):
        if not self.load_model():
            self.update_status("[ERROR] Model not loaded. Train first")
            return

        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            self.update_status("[ERROR] Could not open webcam")
            return

        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap.set(cv2.CAP_PROP_FPS, 15)

        recognized = set()
        try:
            while not self.stop_flag:
                ret, frame = cap.read()
                if not ret:
                    self.update_status("[WARNING] Failed to capture frame")
                    time.sleep(0.1)
                    continue

                try:
                    self.frame_queue.put_nowait(frame)
                except queue.Full:
                    continue

                if not self.frame_queue.empty():
                    frame = self.frame_queue.get()
                    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                    gray = clahe.apply(gray)

                    results = self.yolo_model(rgb_frame, conf=0.3, iou=0.7)
                    if not results or not results[0].boxes:
                        cv2.imshow("Attendance System", frame)
                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            self.stop_flag = True
                        continue

                    for box in results[0].boxes:
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        face_gray = gray[y1:y2, x1:x2]
                        if face_gray.size == 0 or face_gray.shape[0] < 10 or face_gray.shape[1] < 10:
                            continue
                        face_gray = cv2.resize(face_gray, (100, 100))

                        try:
                            label_id, confidence = self.recognizer.predict(face_gray)
                            if confidence < self.confidence_threshold:
                                name = self.labels.get(label_id, "Unknown")
                                if name != "Unknown" and name not in recognized:
                                    if self.mark_attendance(name):
                                        recognized.add(name)
                                color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
                                label = f"{name} ({int(confidence)})" if name != "Unknown" else "Unknown"
                            else:
                                color = (0, 0, 255)
                                label = "Unknown"

                            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                            cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                        except Exception as e:
                            self.update_status(f"[ERROR] Recognition error: {e}")

                    cv2.imshow("Attendance System", frame)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        self.stop_flag = True
                        break

                time.sleep(0.02)
        except Exception as e:
            self.update_status(f"[ERROR] Attendance error: {e}")
        finally:
            cap.release()
            cv2.destroyAllWindows()
            self.update_status("[INFO] Attendance stopped")

In [ ]:
def create_dataset_from_picamera(dataset_path, name, num_samples=10, delay=0.5, update_callback=None, auto_capture=False, yolo_model=None):
    if not name:
        if update_callback:
            update_callback("[ERROR] Invalid name")
        return False

    if yolo_model is None:
        if update_callback:
            update_callback("[ERROR] YOLO model not provided")
        return False

    person_dir = os.path.join(dataset_path, name)
    os.makedirs(person_dir, exist_ok=True)

    try:
        camera = PiCamera()
        camera.resolution = (640, 480)
        camera.framerate = 15
        rawCapture = PiRGBArray(camera, size=(640, 480))
        time.sleep(0.1)  # Allow camera to warm up
    except Exception as e:
        if update_callback:
            update_callback(f"[ERROR] Could not initialize PiCamera: {e}")
        return False

    count = 0
    last_capture_time = time.time()

    if update_callback:
        update_callback(f"[INFO] Capturing images for {name}. Press SPACE to capture{' or wait for auto-capture' if auto_capture else ''}, Q to quit...")

    try:
        for frame in camera.capture_continuous(rawCapture, format="bgr", use_video_port=True):
            image = frame.array

            rgb_frame = image.copy()
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            gray = clahe.apply(gray)

            height, width = image.shape[:2]
            center_x, center_y = width // 2, height // 2
            offset = min(width, height) // 4
            cv2.rectangle(image, (center_x - offset, center_y - offset),
                          (center_x + offset, center_y + offset), (255, 255, 0), 2)
            cv2.putText(image, "Position face in box", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(image, f"Capturing: {count}/{num_samples}", (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            cv2.imshow("Capture Face", image)
            key = cv2.waitKey(1) & 0xFF

            should_capture = False
            current_time = time.time()
            if key == ord(' '):
                should_capture = True
            elif auto_capture and (current_time - last_capture_time) >= delay:
                should_capture = True

            if should_capture:
                results = yolo_model(rgb_frame, conf=0.3, iou=0.7)
                if not results or not results[0].boxes:
                    if update_callback:
                        update_callback("[WARNING] No face detected")
                    rawCapture.truncate(0)
                    continue

                for box in results[0].boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    face = gray[y1:y2, x1:x2]
                    if face.size == 0 or face.shape[0] < 10 or face.shape[1] < 10:
                        if update_callback:
                            update_callback("[WARNING] Invalid face region")
                        continue
                    face = cv2.resize(face, (100, 100))
                    file_path = os.path.join(person_dir, f"{count}.jpg")
                    cv2.imwrite(file_path, face)
                    count += 1
                    last_capture_time = current_time
                    if update_callback:
                        update_callback(f"[INFO] Captured: {count}/{num_samples}")
                    break

            rawCapture.truncate(0)

            if key == ord('q'):
                break

    except Exception as e:
        if update_callback:
            update_callback(f"[ERROR] Capture error: {e}")
        return False

    finally:
        camera.close()
        cv2.destroyAllWindows()

    if count > 0:
        if update_callback:
            update_callback(f"[INFO] Captured {count} images for {name}")
        return True
    if update_callback:
        update_callback("[ERROR] No images captured")
    return False

In [ ]:
def add_external_images(dataset_path, name, status_callback=None, yolo_model=None):
    if not name:
        if status_callback:
            status_callback("[ERROR] Invalid name")
        return False

    if yolo_model is None:
        if status_callback:
            status_callback("[ERROR] YOLO model not provided")
        return False

    file_paths = filedialog.askopenfilenames(title="Select Face Images", filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if not file_paths:
        if status_callback:
            status_callback("[INFO] No files selected")
        return False

    person_dir = os.path.join(dataset_path, name)
    os.makedirs(person_dir, exist_ok=True)
    count = len([f for f in os.listdir(person_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
    success_count = 0

    for file_path in file_paths:
        try:
            img = cv2.imread(file_path)
            if img is None or img.size == 0:
                if status_callback:
                    status_callback(f"[ERROR] Invalid image: {os.path.basename(file_path)}")
                continue

            if img.shape[0] > 480 or img.shape[1] > 640:
                img = cv2.resize(img, (320, 240))

            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            gray = clahe.apply(gray)

            results = yolo_model(rgb_img, conf=0.3, iou=0.7)
            if not results or not results[0].boxes:
                if status_callback:
                    status_callback(f"[WARNING] No face in {os.path.basename(file_path)}")
                continue

            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                face = gray[y1:y2, x1:x2]
                if face.size == 0 or face.shape[0] < 10 or face.shape[1] < 10:
                    if status_callback:
                        status_callback(f"[WARNING] Invalid face region in {os.path.basename(file_path)}")
                    continue
                face_resized = cv2.resize(face, (100, 100))
                img_name = os.path.join(person_dir, f"{count}.jpg")
                cv2.imwrite(img_name, face_resized)
                if status_callback:
                    status_callback(f"[INFO] Processed: {os.path.basename(file_path)}")
                count += 1
                success_count += 1
                break

        except Exception as e:
            if status_callback:
                status_callback(f"[ERROR] Failed to process {os.path.basename(file_path)}: {e}")
            continue

    if status_callback:
        status_callback(f"[INFO] Processed {success_count}/{len(file_paths)} images for {name}")
    return success_count > 0

In [ ]:
def show_attendance(attendance_file):
    attendance_window = tk.Toplevel()
    attendance_window.title("Attendance Records")
    attendance_window.geometry("700x500")
    main_frame = tk.Frame(attendance_window)
    main_frame.pack(fill="both", expand=True, padx=10, pady=10)

    control_frame = tk.Frame(main_frame)
    control_frame.pack(fill="x", pady=(0, 10))
    tk.Label(control_frame, text="Filter by date:").pack(side="left", padx=(0, 5))
    date_var = tk.StringVar()
    date_combobox = ttk.Combobox(control_frame, textvariable=date_var, width=15)
    date_combobox.pack(side="left", padx=(0, 10))
    tk.Label(control_frame, text="Filter by name:").pack(side="left", padx=(10, 5))
    name_var = tk.StringVar()
    name_combobox = ttk.Combobox(control_frame, textvariable=name_var, width=15)
    name_combobox.pack(side="left")
    tk.Label(control_frame, text="Search:").pack(side="left", padx=(10, 5))
    search_var = tk.StringVar()
    search_entry = tk.Entry(control_frame, textvariable=search_var, width=15)
    search_entry.pack(side="left")
    tk.Button(control_frame, text="Search", 
             command=lambda: apply_filters(tree, attendance_file, date_var.get(), name_var.get(), search_var.get())).pack(side="left", padx=5)
    tk.Button(control_frame, text="Reset", 
             command=lambda: load_data(tree, attendance_file, date_combobox, name_combobox)).pack(side="left")

    tree_frame = tk.Frame(main_frame)
    tree_frame.pack(fill="both", expand=True)

    try:
        if not os.path.exists(attendance_file) or os.path.getsize(attendance_file) == 0:
            tk.Label(tree_frame, text="No attendance records found", fg="gray").pack(pady=50)
            return

        tree = ttk.Treeview(tree_frame)
        vsb = ttk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
        vsb.pack(side="right", fill="y")
        tree.configure(yscrollcommand=vsb.set)
        hsb = ttk.Scrollbar(tree_frame, orient="horizontal", command=tree.xview)
        hsb.pack(side="bottom", fill="x")
        tree.configure(xscrollcommand=hsb.set)
        tree.pack(side="left", fill="both", expand=True)
        load_data(tree, attendance_file, date_combobox, name_combobox)
        tk.Button(attendance_window, text="Export to CSV", command=lambda: export_data(tree)).pack(pady=10)
    except Exception as e:
        tk.Label(tree_frame, text=f"Error loading data: {e}", fg="red").pack(pady=20)

def load_data(tree, attendance_file, date_combobox=None, name_combobox=None):
    for item in tree.get_children():
        tree.delete(item)
    tree["columns"] = []
    tree["show"] = "headings"

    try:
        df = pd.read_csv(attendance_file)
        columns = list(df.columns)
        tree["columns"] = columns

        for col in columns:
            tree.heading(col, text=col, anchor="w")
            col_width = max(len(str(col)) * 10, df[col].astype(str).str.len().max() * 10)
            tree.column(col, width=min(col_width, 200), anchor="w")

        for _, row in df.iterrows():
            tree.insert("", "end", values=row.tolist())

        if date_combobox is not None and 'Date' in df.columns:
            date_combobox['values'] = [''] + sorted(df['Date'].unique().tolist())
        if name_combobox is not None and 'Name' in df.columns:
            name_combobox['values'] = [''] + sorted(df['Name'].unique().tolist())
    except Exception as e:
        messagebox.showerror("Error", f"Failed to load data: {e}")

def apply_filters(tree, attendance_file, date_filter=None, name_filter=None, search_text=None):
    for item in tree.get_children():
        tree.delete(item)

    try:
        df = pd.read_csv(attendance_file)
        if date_filter:
            df = df[df['Date'] == date_filter]
        if name_filter:
            df = df[df['Name'] == name_filter]
        if search_text:
            mask = pd.DataFrame(False, index=df.index, columns=['match'])
            for col in df.columns:
                mask['match'] |= df[col].astype(str).str.contains(search_text, case=False, na=False)
            df = df[mask['match']]

        for _, row in df.iterrows():
            tree.insert("", "end", values=row.tolist())
    except Exception as e:
        messagebox.showerror("Error", f"Failed to apply filters: {e}")

def export_data(tree):
    file_path = filedialog.asksaveasfilename(defaultextension='.csv', filetypes=[("CSV files", "*.csv"), ("All files", "*.*")], title="Export Attendance Data")
    if not file_path:
        return

    try:
        columns = tree["columns"]
        data = [tree.item(item)['values'] for item in tree.get_children()]
        pd.DataFrame(data, columns=columns).to_csv(file_path, index=False)
        messagebox.showinfo("Export Successful", f"Data exported to {file_path}")
    except Exception as e:
        messagebox.showerror("Export Error", f"Failed to export: {e}")

In [ ]:
def start_gui(system):
    root = Tk()
    root.title("Advanced Face Attendance System")
    root.geometry("800x600")
    try:
        root.iconbitmap("icon.ico")
    except:
        pass

    tab_control = ttk.Notebook(root)
    tab_dataset = ttk.Frame(tab_control)
    tab_recognition = ttk.Frame(tab_control)
    tab_settings = ttk.Frame(tab_control)
    tab_control.add(tab_dataset, text="Dataset Management")
    tab_control.add(tab_recognition, text="Training & Recognition")
    tab_control.add(tab_settings, text="Settings")
    tab_control.pack(expand=1, fill="both")

    status_frame = Frame(root, relief=tk.SUNKEN, bd=1)
    status_frame.pack(side=tk.BOTTOM, fill=tk.X)
    status_label = Label(status_frame, text="Ready", anchor=tk.W, padx=5)
    status_label.pack(side=tk.LEFT, fill=tk.X)

    def update_status(message):
        status_label.config(text=message)
        root.update_idletasks()

    system.set_status_callback(update_status)

    tk.Label(tab_dataset, text="Person Name:").grid(row=0, column=0, padx=10, pady=10, sticky="w")
    name_entry = tk.Entry(tab_dataset, width=30)
    name_entry.grid(row=0, column=1, padx=10, pady=10, sticky="w")
    tk.Label(tab_dataset, text="Number of Samples:").grid(row=1, column=0, padx=10, pady=10, sticky="w")
    samples_var = tk.StringVar(value=str(system.DEFAULT_NUM_SAMPLES))
    samples_entry = tk.Entry(tab_dataset, textvariable=samples_var, width=10)
    samples_entry.grid(row=1, column=1, padx=10, pady=10, sticky="w")
    auto_capture_var = tk.BooleanVar(value=False)
    tk.Checkbutton(tab_dataset, text="Auto-capture faces", variable=auto_capture_var).grid(row=1, column=2, padx=10, pady=10, sticky="w")

    def capture_webcam():
        try:
            num_samples = int(samples_var.get())
            if num_samples <= 0:
                update_status("[ERROR] Positive samples required")
                return
            create_dataset_from_webcam(system.DATASET_PATH, name_entry.get().strip(), num_samples, 0.5, update_status, auto_capture=auto_capture_var.get(), yolo_model=system.yolo_model)
        except ValueError:
            update_status("[ERROR] Invalid samples number")

    tk.Button(tab_dataset, text="Capture via Webcam", width=20, command=capture_webcam).grid(row=2, column=0, padx=10, pady=10)
    tk.Button(tab_dataset, text="Add External Images", width=20, 
              command=lambda: add_external_images(system.DATASET_PATH, name_entry.get().strip(), update_status, yolo_model=system.yolo_model)).grid(row=2, column=1, padx=10, pady=10)
    tk.Button(tab_dataset, text="View Dataset", width=20, 
              command=lambda: os.startfile(system.DATASET_PATH) if os.name == 'nt' else os.system(f'xdg-open "{system.DATASET_PATH}"')).grid(row=3, column=0, padx=10, pady=10)

    def delete_person():
        name = name_entry.get().strip()
        if not name:
            update_status("[ERROR] Enter a name")
            return
        person_dir = os.path.join(system.DATASET_PATH, name)
        if os.path.exists(person_dir):
            try:
                shutil.rmtree(person_dir)
                update_status(f"[INFO] Deleted dataset for {name}")
            except Exception as e:
                update_status(f"[ERROR] Delete failed: {e}")
        else:
            update_status(f"[ERROR] No dataset for {name}")

    tk.Button(tab_dataset, text="Delete Person", width=20, command=delete_person).grid(row=3, column=1, padx=10, pady=10)

    tk.Button(tab_recognition, text="Train Model", width=20, command=system.train_model).grid(row=0, column=0, padx=10, pady=10)
    tk.Button(tab_recognition, text="Start Attendance", width=20, command=system.run_real_time_attendance).grid(row=1, column=0, padx=10, pady=10)
    tk.Button(tab_recognition, text="Stop Attendance", width=20, command=system.stop_attendance).grid(row=1, column=1, padx=10, pady=10)
    tk.Button(tab_recognition, text="View Attendance", width=20, command=lambda: show_attendance(system.ATTENDANCE_FILE)).grid(row=2, column=0, padx=10, pady=10)
    tk.Button(tab_recognition, text="View Log", width=20, 
              command=lambda: os.startfile(system.LOG_FILE) if os.name == 'nt' else os.system(f'xdg-open "{system.LOG_FILE}"')).grid(row=2, column=1, padx=10, pady=10)
    tk.Button(tab_recognition, text="Test Image", width=20, 
              command=lambda: system.test_recognition(filedialog.askopenfilename())).grid(row=3, column=0, padx=10, pady=10)

    tk.Label(tab_settings, text="Confidence Threshold (0-100):").grid(row=0, column=0, padx=10, pady=10, sticky="w")
    threshold_var = tk.StringVar(value=str(system.confidence_threshold))
    threshold_entry = tk.Entry(tab_settings, textvariable=threshold_var, width=10)
    threshold_entry.grid(row=0, column=1, padx=10, pady=10, sticky="w")
    tk.Button(tab_settings, text="Set Threshold", width=20, 
              command=lambda: system.set_confidence_threshold(threshold_var.get())).grid(row=1, column=0, padx=10, pady=10)

    def clear_attendance():
        if messagebox.askyesno("Confirm", "Clear all attendance records?"):
            try:
                if os.path.exists(system.ATTENDANCE_FILE):
                    os.remove(system.ATTENDANCE_FILE)
                    update_status("[INFO] Attendance cleared")
                else:
                    update_status("[INFO] No attendance to clear")
            except Exception as e:
                update_status(f"[ERROR] Clear failed: {e}")

    tk.Button(tab_settings, text="Clear Attendance", width=20, command=clear_attendance).grid(row=2, column=0, padx=10, pady=10)

    def clear_log():
        if messagebox.askyesno("Confirm", "Clear system log?"):
            try:
                if os.path.exists(system.LOG_FILE):
                    open(system.LOG_FILE, 'w').close()
                    update_status("[INFO] Log cleared")
                else:
                    update_status("[INFO] No log to clear")
            except Exception as e:
                update_status(f"[ERROR] Clear failed: {e}")

    tk.Button(tab_settings, text="Clear Log", width=20, command=clear_log).grid(row=2, column=1, padx=10, pady=10)

    for i in range(4):
        tab_dataset.grid_rowconfigure(i, weight=1)
        tab_recognition.grid_rowconfigure(i, weight=1)
        tab_settings.grid_rowconfigure(i, weight=1)
    for i in range(2):
        tab_dataset.grid_columnconfigure(i, weight=1)
        tab_recognition.grid_columnconfigure(i, weight=1)
        tab_settings.grid_columnconfigure(i, weight=1)

    root.mainloop()

In [ ]:
if __name__ == "__main__":
    try:
        attendance_system = CSVAttendanceSystem()
        start_gui(attendance_system)
    except Exception as e:
        print(f"Startup failed: {e}")
        messagebox.showerror("Startup Error", f"Startup failed: {e}")


0: 480x640 1 face, 44.5ms
Speed: 1.7ms preprocess, 44.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 32.5ms
Speed: 1.9ms preprocess, 32.5ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 36.6ms
Speed: 1.3ms preprocess, 36.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 34.2ms
Speed: 1.0ms preprocess, 34.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 33.1ms
Speed: 1.7ms preprocess, 33.1ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 32.0ms
Speed: 1.7ms preprocess, 32.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 32.6ms
Speed: 1.3ms preprocess, 32.6ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 31.0ms
Speed: 1.3ms preprocess, 31.0ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x